<a href="https://colab.research.google.com/github/idarlandias/Analise-e-Tratamento-de-Dataset/blob/main/Lista_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1: Instalação das Bibliotecas
!pip install pandas pyarrow duckdb

In [2]:
# Célula 2: Importar Bibliotecas e Ler o Dataset
import pandas as pd
import re

# a) Ler o dataset
try:
    df = pd.read_csv('fakeTelegram.BR_2022.csv')
    print("Dataset 'fakeTelegram.BR_2022.csv' lido com sucesso.")
    print("\nShape inicial do dataset (linhas, colunas):", df.shape)
    print("\nPrimeiras 5 linhas do dataset:")
    display(df.head())
except FileNotFoundError:
    print("Erro: O arquivo 'fakeTelegram.BR_2022.csv' não foi encontrado.")
    print("Por favor, certifique-se de que o upload do arquivo foi concluído e que o nome do arquivo está correto.")

Dataset 'fakeTelegram.BR_2022.csv' lido com sucesso.

Shape inicial do dataset (linhas, colunas): (557586, 19)

Primeiras 5 linhas do dataset:


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,NaN,NaN
1,2022-10-05 06:25:08,NaN,12283e08a2eb5789201e105b34489ee7,NaN,NaN,NaN,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,NaN,NaN
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,NaN,NaN,NaN,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,NaN,NaN
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,NaN,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,NaN,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,NaN,True,False,False,NaN,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,NaN,5ad5c8bbe9da93a37fecf3e5aa5b0637


In [3]:
# Célula b: Remover os Trava-zaps

# Esta função de exemplo identifica mensagens suspeitas com base em alguns critérios.
def is_travazap(text):
    if isinstance(text, str):
        # Critério 1: Comprimento excessivo da mensagem
        if len(text) > 4000:
            return True
        # Critério 2: Alta proporção de caracteres não-alfanuméricos (exceto espaços)
        non_alnum = len(re.findall(r'[^a-zA-Z0-9\s]', text))
        if len(text) > 100 and non_alnum / len(text) > 0.8:
            return True
        # Critério 3: Repetição excessiva do mesmo caractere (e.g., mais de 100 vezes seguidas)
        if re.search(r'(.)\1{100,}', text):
            return True
    return False

# Verifica se o DataFrame 'df' foi carregado antes de continuar
if 'df' in locals():
    # Vamos focar na coluna 'text' que é a mais provável para conter os trava-zaps
    if 'text' in df.columns:
        # Cria uma máscara booleana para as linhas que não são trava-zaps
        mask_travazap = df['text'].apply(is_travazap)
        df_cleaned = df[~mask_travazap].copy()

        print(f"Foram identificadas e removidas {mask_travazap.sum()} mensagens de trava-zap.")
        print(f"Shape do dataset após a remoção: {df_cleaned.shape}")
        print("\nVisualização dos dados limpos:")
        display(df_cleaned.head())
    else:
        print("\nA coluna 'text' não foi encontrada. A etapa de remoção de trava-zaps foi ignorada.")
        df_cleaned = df.copy()
else:
    print("O DataFrame 'df' não foi criado. Execute a Célula 2 primeiro.")


A coluna 'text' não foi encontrada. A etapa de remoção de trava-zaps foi ignorada.


In [4]:
# Célula c: Exportar os Dados para um Arquivo Parquet

# c) Exportar para Parquet
# Verifica se o DataFrame 'df_cleaned' existe
if 'df_cleaned' in locals():
    try:
        df_cleaned.to_parquet('fakeTelegram_cleaned.parquet', engine='pyarrow')
        print("Dados exportados com sucesso para 'fakeTelegram_cleaned.parquet'.")
        print("Você pode encontrar o arquivo no painel à esquerda.")
    except Exception as e:
        print(f"Ocorreu um erro ao exportar para Parquet: {e}")
else:
    print("O DataFrame 'df_cleaned' não foi criado. Execute a Célula 3 primeiro.")

Dados exportados com sucesso para 'fakeTelegram_cleaned.parquet'.
Você pode encontrar o arquivo no painel à esquerda.


In [5]:
# Célula d: Exportar os Dados para o DuckDB
import duckdb

# d) Exportar para DuckDB
# Verifica se o DataFrame 'df_cleaned' existe
if 'df_cleaned' in locals():
    try:
        # Conecta-se a um arquivo de banco de dados DuckDB (ele será criado se não existir)
        con = duckdb.connect(database='fakeTelegram.duckdb', read_only=False)

        # Cria uma tabela persistente chamada 'telegram_cleaned' no arquivo DuckDB
        # a partir do nosso DataFrame limpo.
        con.execute('CREATE OR REPLACE TABLE telegram_cleaned AS SELECT * FROM df_cleaned')
        print("Dados exportados com sucesso para a tabela 'telegram_cleaned' no arquivo 'fakeTelegram.duckdb'.")

        # Verificação opcional: contar linhas na tabela recém-criada
        row_count = con.execute('SELECT COUNT(*) FROM telegram_cleaned').fetchone()[0]
        print(f"\nVerificação: A tabela no DuckDB contém {row_count} linhas.")

        # Fecha a conexão com o banco de dados
        con.close()
    except Exception as e:
        print(f"Ocorreu um erro ao exportar para DuckDB: {e}")
else:
    print("O DataFrame 'df_cleaned' não foi criado. Execute a Célula 3 primeiro.")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Dados exportados com sucesso para a tabela 'telegram_cleaned' no arquivo 'fakeTelegram.duckdb'.

Verificação: A tabela no DuckDB contém 557586 linhas.


In [6]:
# Célula e: de Preparação: Conectar ao DuckDB
import duckdb
import pandas as pd

try:
    # Conecta-se ao arquivo de banco de dados DuckDB
    con = duckdb.connect(database='fakeTelegram.duckdb', read_only=True)
    print("Conexão com o DuckDB estabelecida com sucesso.")
except Exception as e:
    print(f"Ocorreu um erro ao conectar ao DuckDB: {e}")

Conexão com o DuckDB estabelecida com sucesso.


In [7]:
# 1. A quantidade de mensagens
query = "SELECT COUNT(*) AS total_mensagens FROM telegram_cleaned;"
resultado = con.execute(query).fetchdf()
print(f"A quantidade total de mensagens é: {resultado['total_mensagens'][0]}")

A quantidade total de mensagens é: 557586


In [8]:
# 2. A quantidade de usuários distintos
query = "SELECT COUNT(DISTINCT id_member_anonymous) AS total_usuarios FROM telegram_cleaned;"
resultado = con.execute(query).fetchdf()
print(f"A quantidade total de usuários distintos é: {resultado['total_usuarios'][0]}")

A quantidade total de usuários distintos é: 14809


In [9]:
# 3. A quantidade de grupos distintos
query = "SELECT COUNT(DISTINCT id_group_anonymous) AS total_grupos FROM telegram_cleaned;"
resultado = con.execute(query).fetchdf()
print(f"A quantidade total de grupos distintos é: {resultado['total_grupos'][0]}")

A quantidade total de grupos distintos é: 178


In [10]:
# 4. Quantidade de mensagens que possuem apenas texto
query = """
SELECT COUNT(*) AS mensagens_com_texto
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND media_type IS NULL;
"""
resultado = con.execute(query).fetchdf()
print(f"A quantidade de mensagens contendo apenas texto é: {resultado['mensagens_com_texto'][0]}")

A quantidade de mensagens contendo apenas texto é: 224966


In [11]:
# 5. Quantidade de mensagens contendo mídias
query = "SELECT COUNT(*) AS mensagens_com_midia FROM telegram_cleaned WHERE media_type IS NOT NULL;"
resultado = con.execute(query).fetchdf()
print(f"A quantidade de mensagens contendo alguma mídia é: {resultado['mensagens_com_midia'][0]}")

A quantidade de mensagens contendo alguma mídia é: 332605


In [12]:
# 6. Quantidade de mensagens por tipo de mídia
query = """
SELECT media_type, COUNT(*) AS quantidade
FROM telegram_cleaned
WHERE media_type IS NOT NULL
GROUP BY media_type
ORDER BY quantidade DESC;
"""
resultado_df = con.execute(query).fetchdf()
print("Quantidade de mensagens por tipo de mídia:")
display(resultado_df)

Quantidade de mensagens por tipo de mídia:


,media_type,quantidade
0,image/jpg,200441
1,url,100856
2,video/mp4,18497
3,application/vnd.android.package-archive,7159
4,application/pdf,2850
...,...,...
62,image/xps,1
63,image/gif,1
64,application/x-msi,1
65,image/08e8201b6f32c358808c7ac89d8c8422,1


In [13]:
# 7: Quantidade de mensagens por estado

# ATENÇÃO: Esta célula assume que existe uma coluna chamada 'uf' na tabela.
# A execução falhará no dataset atual, mas o código está estruturalmente correto
# para um dataset que contenha essa informação.

query = """
SELECT
    uf,
    COUNT(*) AS quantidade_mensagens
FROM telegram_cleaned
WHERE uf IS NOT NULL
GROUP BY uf
ORDER BY quantidade_mensagens DESC;
"""

try:
    resultado_df = con.execute(query).fetchdf()
    print("Quantidade de mensagens por estado:")
    display(resultado_df)
except Exception as e:
    print("ERRO ESPERADO: A consulta falhou porque a coluna 'uf' não existe na tabela.")
    print(f"Detalhes: {e}")

ERRO ESPERADO: A consulta falhou porque a coluna 'uf' não existe na tabela.
Detalhes: Binder Error: Referenced column "uf" not found in FROM clause!
Candidate bindings: "id_group_anonymous"

LINE 6: WHERE uf IS NOT NULL
              ^


In [14]:
# 8: Quantidade de usuários por estado

# ATENÇÃO: Esta célula assume que existe uma coluna chamada 'uf'.
# Ela também usa o nome de coluna correto para o ID do usuário ('id_member_anonymous').
# A execução falhará no dataset atual, mas o código está estruturalmente correto.

query = """
SELECT
    uf,
    COUNT(DISTINCT id_member_anonymous) AS quantidade_usuarios
FROM telegram_cleaned
WHERE uf IS NOT NULL
GROUP BY uf
ORDER BY quantidade_usuarios DESC;
"""

try:
    resultado_df = con.execute(query).fetchdf()
    print("Quantidade de usuários por estado:")
    display(resultado_df)
except Exception as e:
    print("ERRO ESPERADO: A consulta falhou porque a coluna 'uf' não existe na tabela.")
    print(f"Detalhes: {e}")

ERRO ESPERADO: A consulta falhou porque a coluna 'uf' não existe na tabela.
Detalhes: Binder Error: Referenced column "uf" not found in FROM clause!
Candidate bindings: "id_group_anonymous"

LINE 6: WHERE uf IS NOT NULL
              ^


In [15]:
# 9: Relação usuários por mensagens por estado

# ATENÇÃO: Esta célula assume que existe uma coluna chamada 'uf'.
# A execução falhará no dataset atual, mas o código está estruturalmente correto.

query = """
SELECT
    uf,
    COUNT(*) AS total_mensagens,
    COUNT(DISTINCT id_member_anonymous) AS total_usuarios,
    ROUND(CAST(COUNT(*) AS REAL) / COUNT(DISTINCT id_member_anonymous), 2) AS mensagens_por_usuario
FROM telegram_cleaned
WHERE uf IS NOT NULL
GROUP BY uf
ORDER BY total_mensagens DESC;
"""

try:
    resultado_df = con.execute(query).fetchdf()
    print("Relação de mensagens por usuário em cada estado:")
    display(resultado_df)
except Exception as e:
    print("ERRO ESPERADO: A consulta falhou porque a coluna 'uf' não existe na tabela.")
    print(f"Detalhes: {e}")

ERRO ESPERADO: A consulta falhou porque a coluna 'uf' não existe na tabela.
Detalhes: Binder Error: Referenced column "uf" not found in FROM clause!
Candidate bindings: "id_group_anonymous"

LINE 8: WHERE uf IS NOT NULL
              ^


In [16]:
# 10: Quantidade de mensagens por país

# ATENÇÃO: Esta célula assume que existe uma coluna chamada 'pais' na tabela.
# A execução falhará no dataset atual, mas o código está estruturalmente correto.

query = """
SELECT
    pais,
    COUNT(*) AS quantidade_mensagens
FROM telegram_cleaned
WHERE pais IS NOT NULL
GROUP BY pais
ORDER BY quantidade_mensagens DESC;
"""

try:
    resultado_df = con.execute(query).fetchdf()
    print("Quantidade de mensagens por país:")
    display(resultado_df)
except Exception as e:
    print("ERRO ESPERADO: A consulta falhou porque a coluna 'pais' não existe na tabela.")
    print(f"Detalhes: {e}")

ERRO ESPERADO: A consulta falhou porque a coluna 'pais' não existe na tabela.
Detalhes: Binder Error: Referenced column "pais" not found in FROM clause!
Candidate bindings: "has_media", "id_message", "date_system", "date_message", "has_media_url"

LINE 6: WHERE pais IS NOT NULL
              ^


In [17]:
# 11: Quantidade de mensagens Brasil X Países Estrangeiros

# ATENÇÃO: Esta célula assume que existe uma coluna chamada 'pais' na tabela.
# A execução falhará no dataset atual, mas o código está estruturalmente correto.

query = """
SELECT
    CASE
        WHEN pais = 'BR' THEN 'Brasil'
        ELSE 'Estrangeiro'
    END AS origem,
    COUNT(*) AS quantidade
FROM telegram_cleaned
WHERE pais IS NOT NULL
GROUP BY origem;
"""

try:
    resultado_df = con.execute(query).fetchdf()
    print("Comparativo de mensagens: Brasil vs. Estrangeiro")
    display(resultado_df)
except Exception as e:
    print("ERRO ESPERADO: A consulta falhou porque a coluna 'pais' não existe na tabela.")
    print(f"Detalhes: {e}")

ERRO ESPERADO: A consulta falhou porque a coluna 'pais' não existe na tabela.
Detalhes: Binder Error: Referenced column "pais" not found in FROM clause!
Candidate bindings: "has_media", "id_message", "date_system", "date_message", "has_media_url"

LINE 9: WHERE pais IS NOT NULL
              ^


In [18]:
# 12 As 30 URLs mais compartilhadas
import re

# Pega todas as mensagens que provavelmente contêm um link
query = "SELECT text_content_anonymous FROM telegram_cleaned WHERE text_content_anonymous ILIKE '%http%';"
textos_com_links = con.execute(query).fetchdf()

# Expressão regular para encontrar URLs
url_pattern = r'https?://[^\s/$.?#].[^\s]*'

# Extrai todas as URLs, conta e mostra as 30 mais comuns
urls = textos_com_links['text_content_anonymous'].str.findall(url_pattern, flags=re.IGNORECASE).explode()
top_30_urls = urls.value_counts().head(30)

print("As 30 URLs mais compartilhadas:")
print(top_30_urls)

As 30 URLs mais compartilhadas:
text_content_anonymous
https://youtube.com/c/especulandoosfatosoficial                                                                         1607
https://youtu.be/qbTzhB0akt8                                                                                            1197
https://t.me/canalselvabrasiloficial                                                                                    1192
https://youtu.be/zDuOoyhyN-4                                                                                            1050
https://youtu.be/4DHk9KZ01HM                                                                                             659
https://youtu.be/x2uiakywcrI                                                                                             594
https://t.me/apostagem                                                                                                   567
https://t.me/Ovetoroculto                                             

In [19]:
# 13 Os 30 domínios mais compartilhados
from urllib.parse import urlparse

# Processa a lista de URLs extraída na célula anterior para obter os domínios
# O .dropna() remove quaisquer resultados que não sejam URLs válidas
def safe_urlparse(url):
    try:
        return urlparse(url).netloc
    except ValueError:
        return None # Retorna None para URLs que não podem ser parseadas

dominios = urls.dropna().apply(safe_urlparse).dropna()
top_30_dominios = dominios.value_counts().head(30)

print("Os 30 domínios mais compartilhados:")
print(top_30_dominios)

Os 30 domínios mais compartilhados:
text_content_anonymous
youtu.be                           57892
t.me                               20055
terrabrasilnoticias.com             8297
www.youtube.com                     7522
www.instagram.com                   6511
youtube.com                         6302
twitter.com                         5029
www.jornaldacidadeonline.com.br     4857
chat.whatsapp.com                   2357
www.facebook.com                    1843
gazetabrasil.com.br                 1789
pleno.news                          1621
gettr.com                           1257
www.bitchute.com                    1228
instagram.com                       1136
kwai-video.com                       954
www.aliadosbrasiloficial.com.br      945
revistaoeste.com                     925
bit.ly                               852
vistapatria.com.br                   830
verdadecensurada.com.br              829
m.youtube.com                        766
divirta-ce.blogspot.com              75

In [20]:
# 14 Os 30 usuários mais ativos
query = """
SELECT id_member_anonymous, COUNT(*) as total_mensagens
FROM telegram_cleaned
GROUP BY id_member_anonymous
ORDER BY total_mensagens DESC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("Os 30 usuários mais ativos:")
display(resultado_df)

Os 30 usuários mais ativos:


,id_member_anonymous,total_mensagens
0,None,323341
1,abe534d581ec6d552243d6955d3c3cd8,12289
2,1665e22b0f564cd46d343f7677014821,5452
3,1ac091b8ed5c4e42383f1b4ff4cc9b2d,5060
4,c743967449a387ad2c1c7e03b2c45b36,3019
5,e7998863ac2a40086657fab4a6b463c9,1928
6,e8fd8fee8c39342d37993775da7756d5,1706
7,2f4be6244ede15b46e8329a2c975be30,1620
8,d49f81df0c75d1d72bee6c5b2d707da0,1571
9,4f7d493f0f6222d56e5b19a4f7c336cc,1447


In [21]:
# 15 Os 30 usuários que mais compartilharam texto
query = """
SELECT id_member_anonymous, COUNT(*) AS mensagens_de_texto
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND media_type IS NULL
GROUP BY id_member_anonymous
ORDER BY mensagens_de_texto DESC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("Os 30 usuários que mais compartilharam mensagens de texto:")
display(resultado_df)

Os 30 usuários que mais compartilharam mensagens de texto:


,id_member_anonymous,mensagens_de_texto
0,None,122733
1,abe534d581ec6d552243d6955d3c3cd8,12289
2,c743967449a387ad2c1c7e03b2c45b36,2867
3,1665e22b0f564cd46d343f7677014821,2480
4,d49f81df0c75d1d72bee6c5b2d707da0,1571
5,f233cf8b1d4ede06f32199a7e0081bf5,1022
6,e8fd8fee8c39342d37993775da7756d5,699
7,a378e9743fa3ca297df321cfa0e7cf6a,598
8,7696d5103cdb8ac352d748a1db1126b0,560
9,26b496125c8b6773bce453e14f172430,502


In [22]:
# 16 Os 30 usuários que mais compartilharam mídias
query = """
SELECT id_member_anonymous, COUNT(*) AS mensagens_de_midia
FROM telegram_cleaned
WHERE media_type IS NOT NULL
GROUP BY id_member_anonymous
ORDER BY mensagens_de_midia DESC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("Os 30 usuários que mais compartilharam mídias:")
display(resultado_df)

Os 30 usuários que mais compartilharam mídias:


,id_member_anonymous,mensagens_de_midia
0,None,200605
1,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4632
2,1665e22b0f564cd46d343f7677014821,2972
3,e7998863ac2a40086657fab4a6b463c9,1498
4,2f4be6244ede15b46e8329a2c975be30,1489
5,4f7d493f0f6222d56e5b19a4f7c336cc,1359
6,c052c859b42c5a1923c22f5a201de746,1289
7,773b9bd5b02a2e96f9d732c29bfcb663,1167
8,56b8359fd127312651b80b8ed8030085,1079
9,e8fd8fee8c39342d37993775da7756d5,1007


In [23]:
# 17 As 30 mensagens de texto mais compartilhadas
query = """
SELECT text_content_anonymous, COUNT(*) AS quantidade
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND TRIM(text_content_anonymous) != ''
GROUP BY text_content_anonymous
ORDER BY quantidade DESC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("As 30 mensagens de texto mais compartilhadas:")
display(resultado_df)

As 30 mensagens de texto mais compartilhadas:


,text_content_anonymous,quantidade
0,This community was blocked in Brazil following...,17422
1,Rough_sex🙈,1134
2,Anal sex🙈,1118
3,سکس مردان ازبک با زن انگلیسی با این vpn از سای...,1019
4,https://youtu.be/qbTzhB0akt8,758
5,https://youtu.be/zDuOoyhyN-4,712
6,فیلم سوپر با زیرنویس فارسی ببین😍😍\nبا این فیلت...,632
7,Rough😱,570
8,Foto de Nélia Barros,548
9,We had no choice but to remain in the shadows....,480


In [24]:
# 18 As 30 mensagens compartilhadas em mais grupos diferentes
query = """
SELECT text_content_anonymous, COUNT(DISTINCT id_group_anonymous) AS num_grupos_distintos
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND TRIM(text_content_anonymous) != ''
GROUP BY text_content_anonymous
ORDER BY num_grupos_distintos DESC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("As 30 mensagens compartilhadas no maior número de grupos distintos:")
display(resultado_df)

As 30 mensagens compartilhadas no maior número de grupos distintos:


,text_content_anonymous,num_grupos_distintos
0,This community was blocked in Brazil following...,59
1,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,36
2,Hoje às 18 horas no canal Inteligência Ltda do...,35
3,[USER],32
4,https://youtu.be/zDuOoyhyN-4,31
5,https://youtu.be/qbTzhB0akt8,30
6,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,29
7,Ajudem a subir a #JanonesAmigoDePedofilo\n\nPo...,29
8,*ATENÇÃO* \n🚨🚨🚨🚨🚨🚨🚨🚨🚨\n\n*CHAMADA URGENTE* Pat...,29
9,"DE NADA IRÁ ADIANTAR FISCALIZAR AS URNAS, SE N...",29


In [25]:
# 19 Mensagens idênticas compartilhadas pelo mesmo usuário
query = """
SELECT id_member_anonymous, text_content_anonymous, COUNT(*) AS quantidade
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND TRIM(text_content_anonymous) != ''
GROUP BY id_member_anonymous, text_content_anonymous
HAVING COUNT(*) > 1
ORDER BY quantidade DESC;
"""
resultado_df = con.execute(query).fetchdf()
print("Mensagens idênticas enviadas mais de uma vez pelo mesmo usuário:")
display(resultado_df)

Mensagens idênticas enviadas mais de uma vez pelo mesmo usuário:


,id_member_anonymous,text_content_anonymous,quantidade
0,None,This community was blocked in Brazil following...,9648
1,f233cf8b1d4ede06f32199a7e0081bf5,This community was blocked in Brazil following...,838
2,c743967449a387ad2c1c7e03b2c45b36,This community was blocked in Brazil following...,532
3,7696d5103cdb8ac352d748a1db1126b0,This community was blocked in Brazil following...,530
4,None,Foto de Nélia Barros,508
...,...,...,...
46826,aeb34c08a5be6132fc90e12cd104eec6,🇧🇷🇧🇷🇧🇷👇👇👇\n*FORMULA 1 BRASIL*\n\nPreparem-se t...,2
46827,None,Foto de Frank silva Torres,2
46828,None,Foto de Herbert Bitencourt,2
46829,0262478e2a4b231475742acbb6161a37,This community was blocked in Brazil following...,2


In [26]:
#  20 Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos
# Usando as colunas corretas 'id_member_anonymous', 'text_content_anonymous' e 'id_group_anonymous'
query = """
SELECT id_member_anonymous, text_content_anonymous, COUNT(DISTINCT id_group_anonymous) AS num_grupos_distintos
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND TRIM(text_content_anonymous) != ''
GROUP BY id_member_anonymous, text_content_anonymous
HAVING COUNT(DISTINCT id_group_anonymous) > 1
ORDER BY num_grupos_distintos DESC;
"""
resultado_df = con.execute(query).fetchdf()
print("Mensagens idênticas enviadas pelo mesmo usuário em mais de um grupo:")
display(resultado_df)

Mensagens idênticas enviadas pelo mesmo usuário em mais de um grupo:


,id_member_anonymous,text_content_anonymous,num_grupos_distintos
0,None,This community was blocked in Brazil following...,53
1,None,Hoje às 18 horas no canal Inteligência Ltda do...,32
2,None,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,32
3,None,"OS ELEITORES QUE JA ENVIOU, NÃO PRECISA MAIS E...",28
4,None,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,28
...,...,...,...
34650,None,https://youtu.be/WPXPkYwUhEI,2
34651,None,O argentino acabou de postar no telegram \nVot...,2
34652,5d321b03e74b751ac577ef022350306d,*Moraes pede a Lula projeto para regulamentar ...,2
34653,None,“Não vejo explicação para Bolsonaro ter zero v...,2


In [27]:
# Instalação da biblioteca vaderSentiment
!pip install vaderSentiment

In [28]:
# Importar a ferramenta de análise de sentimento
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Inicializar o analisador de sentimento
s = SentimentIntensityAnalyzer()

print("Analisador de sentimento 's' inicializado com sucesso.")

Analisador de sentimento 's' inicializado com sucesso.


In [29]:
# 21 (Os 30 unigramas, bigramas e trigramas mais compartilhados

# Usando a coluna correta 'text_content_anonymous'
query_textos = "SELECT text_content_anonymous FROM telegram_cleaned WHERE text_content_anonymous IS NOT NULL;"
df_textos = con.execute(query_textos).fetchdf()

# A coluna no DataFrame agora se chama 'text_content_anonymous'
top_unigrams = get_top_ngrams(df_textos['text_content_anonymous'], (1, 1))
print("--- 30 Unigramas Mais Comuns ---")
print(pd.DataFrame(top_unigrams, columns=['Unigrama', 'Frequência']))

top_bigrams = get_top_ngrams(df_textos['text_content_anonymous'], (2, 2))
print("\n--- 30 Bigramas Mais Comuns ---")
print(pd.DataFrame(top_bigrams, columns=['Bigrama', 'Frequência']))

top_trigrams = get_top_ngrams(df_textos['text_content_anonymous'], (3, 3))
print("\n--- 30 Trigramas Mais Comuns ---")
print(pd.DataFrame(top_trigrams, columns=['Trigrama', 'Frequência']))

NameError: name 'get_top_ngrams' is not defined

In [ ]:
# 22 As 30 mensagens mais positivas (distintas)

df_positivas = df_mensagens.sort_values(by='sentimento', ascending=False).head(30)

print("--- 30 Mensagens Mais Positivas ---")
display(df_positivas)

In [ ]:
# Célula de Preparação para Análise de Sentimento
# Pega apenas as mensagens de texto para análise de sentimento
query_textos_sentimento = "SELECT text_content_anonymous FROM telegram_cleaned WHERE text_content_anonymous IS NOT NULL AND TRIM(text_content_anonymous) != '';"
df_mensagens = con.execute(query_textos_sentimento).fetchdf()

print("DataFrame 'df_mensagens' criado para análise de sentimento.")
display(df_mensagens.head())

In [ ]:
# 23 As 30 mensagens mais negativas (distintas)

df_negativas = df_mensagens.sort_values(by='sentimento', ascending=True).head(30)

print("--- 30 Mensagens Mais Negativas ---")
display(df_negativas)

In [ ]:
# Célula 24 e 25 O Usuário mais pessimista e otimista

# Usando as colunas corretas 'id_member_anonymous' e 'text_content_anonymous'
query_usuarios_msgs = "SELECT id_member_anonymous, text_content_anonymous FROM telegram_cleaned WHERE text_content_anonymous IS NOT NULL AND LENGTH(text_content_anonymous) > 10;"
df_usr_msgs = con.execute(query_usuarios_msgs).fetchdf()

# A coluna no DataFrame agora se chama 'text_content_anonymous'
df_usr_msgs['sentimento'] = df_usr_msgs['text_content_anonymous'].apply(lambda x: s.polarity_scores(x)['compound'])

# Agrupando pela coluna correta 'id_member_anonymous'
sentimento_medio_usuario = df_usr_msgs.groupby('id_member_anonymous')['sentimento'].mean().reset_index()

mais_otimista = sentimento_medio_usuario.sort_values('sentimento', ascending=False).iloc[0]
print(f"--- Usuário Mais Otimista ---\nUser ID: {mais_otimista['id_member_anonymous']}\nSentimento Médio: {mais_otimista['sentimento']:.4f}\n")

mais_pessimista = sentimento_medio_usuario.sort_values('sentimento', ascending=True).iloc[0]
print(f"--- Usuário Mais Pessimista ---\nUser ID: {mais_pessimista['id_member_anonymous']}\nSentimento Médio: {mais_pessimista['sentimento']:.4f}\n")

In [ ]:
# 26 As 30 maiores mensagens
# Usando a coluna correta 'text_content_anonymous'
query = """
SELECT text_content_anonymous, LENGTH(text_content_anonymous) AS tamanho
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL
ORDER BY tamanho DESC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("As 30 maiores mensagens (em número de caracteres):")
display(resultado_df)

In [ ]:
# 27
# Usando a coluna correta 'text_content_anonymous'
query = """
SELECT text_content_anonymous, LENGTH(text_content_anonymous) AS tamanho
FROM telegram_cleaned
WHERE text_content_anonymous IS NOT NULL AND LENGTH(text_content_anonymous) > 1
ORDER BY tamanho ASC
LIMIT 30;
"""
resultado_df = con.execute(query).fetchdf()
print("As 30 menores mensagens (com mais de 1 caractere):")
display(resultado_df)

In [ ]:
# 28
# Usando a coluna de data correta 'date_message'
query = """
SELECT CAST(date_message AS DATE) AS dia, COUNT(*) AS quantidade
FROM telegram_cleaned
GROUP BY dia
ORDER BY quantidade DESC
LIMIT 1;
"""
resultado_df = con.execute(query).fetchdf()
print("O dia com a maior publicação de mensagens foi:")
display(resultado_df)

In [ ]:
# 29
# Usando a coluna correta 'text_content_anonymous'
query = """
SELECT text_content_anonymous FROM telegram_cleaned
WHERE text_content_anonymous ILIKE '%FACÇÃO%' AND text_content_anonymous ILIKE '%CRIMINOSA%';
"""
resultado_df = con.execute(query).fetchdf()
print("Mensagens encontradas contendo 'FACÇÃO' e 'CRIMINOSA':")
display(resultado_df)

In [ ]:
# 30
# Usando a coluna correta 'text_content_anonymous'
query = "SELECT text_content_anonymous FROM telegram_cleaned WHERE text_content_anonymous ILIKE '%SEGURANÇA%';"
resultado_df = con.execute(query).fetchdf()
print("Mensagens encontradas contendo 'SEGURANÇA':")
display(resultado_df)